# Quasi-dynamic spring slider with 1 state variable

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve
from scipy.spatial.distance import pdist
from scipy.stats import linregress

### Parameters

In [ ]:
### Rate-and-state
# reference friction coefficient
fr = 0.6 
# reference slip velocity (m/s)
Vr = 1e-6
# direct effect parameter 
#a = 0.015
#a = 0.020
# state evolution parameter
b = 0.018
# state evolution distance (m)
dc = 1e-2

### Elasticity
# density (g/cm^3)
rho = 2.54
# shear velocity (m/s)
c = 2.5
# radiation damping (MPa*s/m)
eta = rho*c/2
# normal stress (MPa)
sigma = 50
# initial shear stress (MPa)
tau0 = 0.5793*sigma
# calculate critical spring constant (MPa/m)
kcr = sigma*abs(b-a)/dc
# spring constante (MPa/m)
k = 2*kcr
# loading: shear stress increases at constant rate in 'absence' of slip
oneyear = np.pi*1e7
# delta tau (MPa/s)
dtau = 10/oneyear



#### calculate bifurcation values with radiation damping
B = sigma/dc * (a-b)
C = dtau*eta/dc

kcr1 = (-B+np.sqrt(B**2-4*C))/2
kcr2 = (-B-np.sqrt(B**2-4*C))/2
print(kcr1)
print(kcr2)

#k = kcr/10
k=1.5
################


### Non-dimensional groups (combos of above)
eps1 = dtau/(k*Vr) 
eps2 = sigma*fr/(eta*Vr)
eps3 = a/fr
eps4 = eta*Vr/(k*dc*fr)


In [ ]:
dtau*eta/dc

In [ ]:
kcr

### Set up for solve

In [ ]:
### Time (s)
# max simulation time (s)
tmax = 2*oneyear
# initial time (s)
t0 = 0


### Initial conditions
# Initial slip velocity (m/s)
v0 = 1e-9
# Initial friciton coefficient
f0 = 0.57
# dimension-less versions
x0 = np.log(v0/Vr)
y0 = f0/fr
T0 = t0*k/eta
Tmax = tmax*k/eta
# stack initial values
s0 = np.zeros(2)
s0[0] = x0
s0[1] = y0
t_span = np.array([T0, Tmax])
maxStep = 1e9*k/eta

In [ ]:
def nonDimSliderSystem(T,x):
    # T: non-dimensional time
    # x:
    # x[0] = x = ln(v/Vr)
    # x[1] = y = f/fr
    
    # a common factor:
    cf = eps4*(f0*(1-x[1])+(a-b)*x[0])
    
    #xp[0] = (eps1*np.exp(-x[0]) -1 - eps2*cf)/(1+eps2*eps3*np.exp(-x[0]))
    #xp[1] = (eps1*eps3*np.exp(-x[0])-eps3+np.exp(x[0])*cf)/(1+eps2*eps3*np.exp(-x[0]))
    
    return np.array([ (eps1*np.exp(-x[0]) -1 - eps2*cf)/(1+eps2*eps3*np.exp(-x[0])),
                    (eps1*eps3*np.exp(-x[0])-eps3+np.exp(x[0])*cf)/(1+eps2*eps3*np.exp(-x[0])) ] 
                   )
   
    

### Scipy integrate

In [ ]:
sol=solve_ivp(nonDimSliderSystem, t_span, s0, method='RK45', t_eval=None,
              dense_output=False, events=None, vectorized=True,
              args=None,max_step=maxStep,rtol=1e-4)

In [ ]:
sol.y

### Plot

#### Solutions

In [ ]:
# non-dimensional variables
x = sol.y[0,:]
y = sol.y[1,:]
T = sol.t

# dimensional variables, just to check if solution makes sense
v = Vr*np.exp(x)
f = fr*y
t = eta/k*T

In [ ]:
fig, ax = plt.subplots(nrows = 2, ncols = 2,figsize=(8,8))
ax[0,0].plot(T,x)
ax[0,0].set(xlabel = 'T', ylabel = 'x',title='Nondimensional velocity')
ax[0,1].plot(T,y)
ax[0,1].set(xlabel = 'T', ylabel = 'y',title='Nondimensional friction')
ax[1,0].plot(t,v)
ax[1,0].set(xlabel = 't (s)', ylabel = 'v (m/s)',title='velocity')
ax[1,1].plot(t,f)
ax[1,1].set(xlabel = 't (s)', ylabel = 'f',title='friction')

plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.3, 
                    hspace=0.3)

#### phase diagram

In [ ]:
def nullclines(x):
    # x is the non-dimensional varaible x and is a 1D array here
    # xnull and ynull are y values corresponding to x-nullcline and y-nullcline for these nondimensional variables
    xnull = (a-b)/fr*x + 1/(eps2*eps4*fr)-eps1*np.exp(-x)/(eps2*eps4*fr)+1
    ynull = (a-b)/fr*x - eps3*np.exp(-x)/(eps4*fr)+eps1*eps3/(eps4*fr)*np.exp(-2*x)+1
    
    return xnull, ynull

def nullclines_intersection(x):
    return (eps3-eps1/eps2)*np.exp(-x)+1/eps2-eps1*eps3*np.exp(-2*x)

def plot_vector_field(ax,xrange,yrange,xsol,ysol,nums=50):
    #
    
    # compute vector field
    x = np.linspace(xrange[0],xrange[1],nums)
    y = np.linspace(yrange[0],yrange[1],nums)
    X,Y = np.meshgrid(x,y)
    dx,dy = nonDimSliderSystem(_,np.array([X,Y]))   # '_' place holder for T, not used
    
    # magnitude of flow in log10
    mag = np.log10(np.sqrt(dx**2+dy**2))
    
    # nullclines
    xnull, ynull = nullclines(x)
    
    # solve for fixed point
    #fpy = fsolve(nullclines_intersection,[0.9,1.1])
    
    c = ax.pcolormesh(X,Y,mag,cmap='viridis')
    c.set_clim(-10,2)
    ax.streamplot(X,Y,dx,dy,color=(0,0,0,.1))
    ax.plot(xsol,ysol,color='black')
    ax.plot(xsol[0],ysol[0], marker='o',markersize=8, markeredgecolor='black',markerfacecolor="blue")
    ax.plot(xsol[-1],ysol[-1], marker='D',markersize=8, markeredgecolor='black',markerfacecolor="red")
    ax.plot(x,xnull,color='red',linewidth=2)
    ax.plot(x,ynull,color='cyan',linewidth=2)
    
    ax.set(xlim=(xrange[0], xrange[1]), ylim=(yrange[0], yrange[1]))
    
    return c,mag

In [ ]:
# solve for fixed point
aa = 1/eps2
bb = eps3-eps1/eps2
cc = eps1*eps3

fp_x1 = np.log((-bb+np.sqrt(bb**2+4*aa*cc))/(2*aa))
fp_x2 = np.log((-bb-np.sqrt(bb**2+4*aa*cc))/(2*aa))
print(fp_x1)
print(fp_x2)
if np.isnan(fp_x1):
    fp_x = fp_x2
else:
    fp_x = fp_x1
    
fp_y = nullclines(fp_x)[0]

In [ ]:
# xrange = [np.min(sol.y[0,:]), np.max(sol.y[0,:])]
# yrange = [np.min(sol.y[1,:]), np.max(sol.y[1,:])]

xrange = [-10, 15]
yrange = [0.9, 1.075]
#yrange = [-150, 150]
fig,ax = plt.subplots(1,1,figsize=(6,4))
c,mag = plot_vector_field(ax,xrange,yrange,x,y,2000)
ax.set(xlabel='x',ylabel='y')
cb = fig.colorbar(c, ax=ax)
cb.set_label('flow magnitude')


### Plot fixed point
# observe graphically where the fixed point is first
# to specify initial trial solutions
#fp_x = fsolve(nullclines_intersection,x0=-6)
#fp_y = nullclines(fp_x)[0]
plt.plot(fp_x,fp_y, marker='*',markersize=20, markeredgecolor='black',markerfacecolor="yellow")


plt.title('VS, $k=1.5 MPa/m$')
### Zoom-in
#plt.axis([5, 15, 0.92, 0.98])
plt.rcParams.update({'font.size': 16})

#### difference between x and y nullclines

In [ ]:
# difference between x and y nullclines
xsamples = np.linspace(-1.8,15,1000)
xsamples2 = np.linspace(10,25,1000)
xnull, ynull = nullclines(xsamples)
xnull2,ynull2 = nullclines(xsamples2)   
fig,ax = plt.subplots(1,2,figsize=(20,8))
ax[0].plot(xsamples,ynull-xnull,color='black')
ax[0].set(xlabel='x',ylabel='ynull-xnull')

ax[1].plot(xsamples2,ynull2-xnull2,color='black')
ax[1].set(xlabel='x',ylabel='ynull-xnull')

In [ ]:
# theoretical seperation as x-> infinity
print(1/(eps2*eps4*fr))

# calculated separation
print(xnull[-1]-ynull[-1])

#### Derivatives of nullclines

In [ ]:
# x value corresponding to inflection point on x-nullcline
-np.log((b-a)*eps2*eps4/eps1)  # for VS, b-a < 0, no turning point

In [ ]:
### can solve this analytically

def dyncdx(x):
    return (a-b)/fr+eps3/(eps4*fr)*np.exp(-x)-(2*eps1*eps3/(eps4*fr))*np.exp(-2*x)

ync_max = fsolve(dyncdx,x0=[-4])
print(ync_max)

In [ ]:
# plot dyncdx
xsamples = np.linspace(-1,10,1000)
dync = dyncdx(xsamples)

fig,ax = plt.subplots(1,1,figsize=(8,8))
ax.plot(xsamples,dync,color='black')
ax.set(xlabel='x',ylabel='dync/dx')

## Jacobian and nature of fixed point

In [ ]:
def jac(x,y):
    bigBracketX = eps1*np.exp(-x)-1-eps2*eps4*(fr*(1-y)+x*(a-b))
    bigBracketY = eps1*eps3*np.exp(-x)-eps3+np.exp(x)*eps4*(fr*(1-y)+x*(a-b))
    
    Xx = -(1+eps2*eps3*(np.exp(-x)))**(-2)*(-eps2*eps3*np.exp(-x))*bigBracketX + \
         (1+eps2*eps3*np.exp(-x))**(-1)*(-eps1*np.exp(-x)-eps2*eps4*(a-b))
        
    Xy = (1+eps2*eps3*np.exp(-x))**(-1) *eps2*eps4*fr
    
    Yx = -(1+eps2*eps3*(np.exp(-x)))**(-2)*(-eps2*eps3*np.exp(-x))*bigBracketY + \
         (1+eps2*eps3*np.exp(-x))**(-1)* \
         (-eps1*eps3*np.exp(-x)+np.exp(x)*eps4*(fr*(1-y)+x*(a-b))+np.exp(x)*eps4*(a-b))
            
    Yy = -(1+eps2*eps3*np.exp(-x))**(-1)*np.exp(x)*eps4*fr

    
    return np.array([[Xx, Xy],
                     [Yx, Yy]])

In [ ]:
jac_fp = jac(fp_x,fp_y)

In [ ]:
jac_fp

In [ ]:
# use eigenvalues to determine nature of fixed point
np.linalg.eigvals(jac_fp)

In [ ]:
np.matrix.trace(jac_fp)

In [ ]:
np.linalg.det(jac_fp)

# Correlation dimension

In [ ]:
def correlation_integral(dist,r_range, N):
    '''
    dist: euclidean norm, without log being taken
    r_range: generated in logspace, but without log being taken
    N: number of total points/states of the system
    '''
     
    dist = np.log10(dist)
    r_range = np.log10(r_range)
    
    Cr = []
    for r in r_range:
        Cr.append(np.count_nonzero(dist <= r)/N**2)
    
    return np.array(Cr)

In [ ]:
xy = list(zip(x,y))
distances = pdist(xy)
N = len(x)

In [ ]:
rmin = np.log10(np.min(distances))
print(rmin)

In [ ]:
rmax = np.log10(np.max(distances))
print(rmax)

In [ ]:
#neighborhood radius
r = np.logspace(-12.5, 1.5, 50)

In [ ]:
C = correlation_integral(distances,r,N)

In [ ]:
C

In [ ]:
ax = plt.figure().add_subplot()
ax.plot(np.log10(r),np.log10(C),'bo')

In [ ]:
fitstart = 4
fitend = -4
r_fit = r[fitstart:fitend]
C_fit = C[fitstart:fitend]
line_fit=linregress(np.log10(r_fit),np.log10(C_fit))
print(line_fit)

In [ ]:
ax = plt.figure().add_subplot()
ax.plot(np.log10(r),np.log10(C),'bo')
ax.plot(np.log10(r_fit),np.log10(C_fit),'r*')
ax.plot(np.log10(r_fit),line_fit.slope*np.log10(r_fit)+line_fit.intercept,color='black')
ax.set(xlabel=r'$log_{10} \: r $',ylabel=r'$log_{10} \: C(r) $')
plt.title('VW, $k=2k_{cr}$')

## 